<a href="https://colab.research.google.com/github/wilsantow/Fast-AI-Deep-Learning-1/blob/master/lesson1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fast AI Deep learning course -part 1(2019) Lesson 1 Experiments

This is  my experiment on lesson 1 of Fast AI's Deep learning course -part 1(2019).

Classifying Cricket Shots(Upper Cut,Straight Drive,Sweep Shot)

I extended the idea of [Nikhil B](https://towardsdatascience.com/fun-with-small-image-data-sets-8c83d95d0159 )(Fast AI student) of identifying a picture of a person playing cicket or baseball to identifying between three types of Cricket Shots.

Every notebook starts with the following three lines; they ensure that any edits to libraries you make are reloaded here automatically, and also that any charts or images displayed are shown in this notebook.

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

We import all the necessary packages. We are going to work with the [fastai V1 library](http://www.fast.ai/2018/10/02/fastai-ai/) which sits on top of [Pytorch 1.0](https://hackernoon.com/pytorch-1-0-468332ba5163). The fastai library provides many useful functions that enable us to quickly and easily build neural networks and train our models.

In [0]:
from fastai.vision import *
from fastai.metrics import error_rate

Since the Dataset Is Small,Batch size is set to 18.

In [0]:
bs = 18


## Looking at the data

The Data for this experiment is small.It was manually downloaded from the web and put together.

The data resides in my Google Drive.

It is mounted to colab to be accesed by the colab.


In [0]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)


In [0]:
path =Path('/content/drive/My Drive/fastai-lsn1')

In [0]:
path.ls()

The first thing we do when we approach a problem is to take a look at the data. We _always_ need to understand very well what the problem is and what the data looks like before we can figure out how to solve it. Taking a look at the data means understanding how the data directories are structured, what the labels are and what some sample images look like.

The main difference between the handling of image classification datasets is the way labels are stored. In this particular dataset, labels are the names of the folders in which the images are stored . 

In [0]:
tfms = get_transforms(do_flip=False)
data = ImageDataBunch.from_folder(path, ds_tfms=tfms, size=200 ,bs=18)

In [0]:
data.show_batch(rows=3, figsize=(10,10))

In [0]:
print(data.classes)
len(data.classes),data.c

## Training: resnet34

Now we will start training our model. We will use a [convolutional neural network](http://cs231n.github.io/convolutional-networks/) backbone and a fully connected head with a single hidden layer as a classifier.  we are building a model which will take images as input and will output the predicted probability for each of the categories (in this case, it will have 3 outputs).

We will train for 4 epochs (4 cycles through all our data).

In [0]:
learn = create_cnn(data, models.resnet34, metrics=error_rate)

In [0]:
learn.model

In [0]:
learn.fit_one_cycle(5)

In [0]:
learn.save('stage-1')

## Results

Let's see what results we have got. 

We will first see which were the categories that the model most confused with one another. We will try to see if what the model predicted was reasonable or not. In this case the mistakes look reasonable (none of the mistakes seems obviously naive). This is an indicator that our classifier is working correctly. 

Furthermore, when we plot the confusion matrix, we can see that the distribution is heavily skewed: the model makes the same mistakes over and over again but it rarely confuses other categories. This suggests that it just finds it difficult to distinguish some specific categories between each other; this is normal behaviour.

In [0]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [0]:
interp.plot_top_losses(9, figsize=(15,11))

In [0]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [0]:
interp.most_confused(min_val=1)

## Unfreezing, fine-tuning, and learning rates

Since our model is working as we expect it to, we will *unfreeze* our model and train some more.

In [0]:
learn.unfreeze()

In [0]:
learn.fit_one_cycle(5)

In [0]:
learn.load('stage-1');

In [0]:
learn.lr_find()

In [0]:
learn.recorder.plot()

In [0]:
#learn.unfreeze()
#learn.fit_one_cycle(7, max_lr=slice(1e-3,1e-8))